In [ ]:
# %load K-means.py
import open3d as o3d
import numpy as np
from copy import deepcopy
from sklearn import cluster


# Define Plane Segmentation Function
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])


# Load a pointcloud
pcd = o3d.io.read_point_cloud("pointcloud1.pcd")
print(pcd)
print(np.asarray(pcd.points))

# Down-sampling：Down-sample the point cloud with a voxel of 0.1
downpcd = pcd.voxel_down_sample(voxel_size=0.1)
print(downpcd)

# Statistical outlier removal
cl, ind = downpcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
display_inlier_outlier(downpcd, ind)
# ‘downpcd_inlier_cloud’ indicates the part of the points to be selected
downpcd_inlier_cloud = downpcd.select_by_index(ind)
print(downpcd_inlier_cloud)

# Plane Segmentation
plane_model, inliers = downpcd_inlier_cloud.segment_plane(distance_threshold=0.2,
                                                          ransac_n=10,
                                                          num_iterations=10000)
[a, b, c, d] = plane_model
# Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0
# Define inlier_cloud and outlier_cloud
inlier_cloud = downpcd_inlier_cloud.select_by_index(inliers)
outlier_cloud = downpcd_inlier_cloud.select_by_index(inliers, invert=True)

# K-means
pcd = o3d.geometry.PointCloud(outlier_cloud)
points = np.array(outlier_cloud.points)
# In the DBSCAN clustering method, the clusters formed by the clusters are about 70,
# thus the number of clusters generated by k-means is chosen to be 60
kmeans = cluster.KMeans(n_clusters=60)
kmeans.fit(points)
labels = kmeans.labels_
max_label = labels.max()
# Display color settings
colors = np.random.randint(0, 255, size=(max(labels) + 1, 3)) / 255
colors = colors[labels]
colors[labels < 0] = 0
pcd_cluster = deepcopy(outlier_cloud)
pcd_cluster.translate([50, 0, 0])
pcd_cluster.colors = o3d.utility.Vector3dVector(colors)

# Finally add the bounding box
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(pcd_cluster)
# Adding bounding boxes to different clusters
for i in range(0, max_label + 1):
    ind = np.where(labels == i)[0]
    cluster_cloud = pcd_cluster.select_by_index(ind)
    aabb = cluster_cloud.get_axis_aligned_bounding_box()
    aabb.color = (1, 0, 0)
    vis.add_geometry(aabb)
vis.run()
